In [6]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import gaussian_kde
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import skew, kurtosis
from tqdm import tqdm

In [28]:
df=pd.read_csv("top_100 accounts.csv")

In [26]:
df.head()

,Account,Timestamp IST,Size USD,Closed PnL,Coin
0,0x8250a7f9fca9c33e68eead8c05c1d58c8fd70d13,2023-11-09 04:17:40,14322.80,0.0,BTC
1,0x8250a7f9fca9c33e68eead8c05c1d58c8fd70d13,2023-11-09 04:18:37,53355.20,0.0,BTC
2,0x8250a7f9fca9c33e68eead8c05c1d58c8fd70d13,2023-11-09 04:22:47,58228.41,0.0,BTC
3,0x8250a7f9fca9c33e68eead8c05c1d58c8fd70d13,2023-11-09 04:27:21,7173.40,0.0,BTC
4,0x8250a7f9fca9c33e68eead8c05c1d58c8fd70d13,2023-11-09 04:27:28,36153.32,0.0,BTC


In [27]:
df.sort_values(['Account', 'Timestamp IST'])

,Account,Timestamp IST,Size USD,Closed PnL,Coin
1617433,0x000000209e0a892842bc06c354e7d097392cbbbb,2024-02-16 02:33:32,1841.62,0.0,MAVIA
1617434,0x000000209e0a892842bc06c354e7d097392cbbbb,2024-02-16 02:33:32,1795.90,0.0,MAVIA
1617435,0x000000209e0a892842bc06c354e7d097392cbbbb,2024-02-16 02:33:32,3472.40,0.0,MAVIA
1617436,0x000000209e0a892842bc06c354e7d097392cbbbb,2024-02-16 02:33:32,906.58,0.0,MAVIA
9594397,0x000000209e0a892842bc06c354e7d097392cbbbb,2024-02-16 02:33:32,1841.62,0.0,MAVIA
...,...,...,...,...,...
11213264,0xffec68dced77787469f1abc902f1d04b9cc09866,2025-06-28 09:50:41,19607.74,205.913044,BTC
11213265,0xffec68dced77787469f1abc902f1d04b9cc09866,2025-06-28 09:50:41,141.49,1.485924,BTC
11213266,0xffec68dced77787469f1abc902f1d04b9cc09866,2025-06-28 09:50:41,1929.47,20.2626,BTC
11213267,0xffec68dced77787469f1abc902f1d04b9cc09866,2025-06-28 09:50:41,9998.96,105.005296,BTC


In [32]:
df = df.copy()
df['Closed PnL'] = df['Closed PnL'].replace('-', np.nan).astype(float)
df['Timestamp IST'] = pd.to_datetime(df['Timestamp IST'])
df = df.sort_values(['Account', 'Timestamp IST'])

# Calculate cumulative PnL
df['Cumulative PnL'] = df.groupby('Account')['Closed PnL'].cumsum()
df['Month'] = df['Timestamp IST'].dt.to_period('M')

In [42]:
def cluster_traders(df, n_clusters=4):
    """
    Cluster traders based on their performance characteristics
    
    Parameters:
    df (pd.DataFrame): Input dataframe containing trader data
    n_clusters (int): Number of clusters to create (default: 4)
    
    Returns:
    pd.DataFrame: DataFrame with cluster assignments and features
    """
    # Feature extraction function
    def extract_features(trader_data):
        # Get all monthly data points
        monthly_data = trader_data.groupby('Month').agg(
            Cumulative_PnL=('Cumulative PnL', 'last'),
            Timestamp=('Timestamp IST', 'last')
        ).reset_index()
        
        # Timeline features
        timeline_features = monthly_data.set_index('Timestamp')['Cumulative_PnL']
        
        # Trend features
        monthly_changes = timeline_features.diff().dropna()
        trend_stability = monthly_changes.std()
        
        # Drawdown features
        cumulative_max = timeline_features.cummax()
        drawdown = cumulative_max - timeline_features
        max_drawdown = drawdown.max()
        recovery_factor = timeline_features.iloc[-1] / max_drawdown if max_drawdown > 0 else 10
        
        # Trade distribution features
        trades = trader_data['Closed PnL'].dropna()
        n_trades = len(trades)
        
        if n_trades > 0:
            wins = trades > 0
            win_rate = wins.mean()
            win_sum = trades[wins].sum()
            loss_sum = trades[~wins].sum()
            profit_factor = win_sum / abs(loss_sum) if loss_sum < 0 else 10
            trade_skew = trades.skew() if n_trades > 2 else 0
            trade_kurt = trades.kurtosis() if n_trades > 3 else 0
        else:
            win_rate = profit_factor = trade_skew = trade_kurt = 0
        
        # Activity features
        trade_density = n_trades / len(monthly_data) if len(monthly_data) > 0 else 0
        
        return {
            'Cumulative_PnL': timeline_features.iloc[-1],  # Last cumulative value
            'Timeline_Length': len(monthly_data),  # Number of months in timeline
            'Trend_Stability': trend_stability,
            'Max_Drawdown': max_drawdown,
            'Recovery_Factor': recovery_factor,
            'Win_Rate': win_rate,
            'Profit_Factor': profit_factor,
            'Trade_Skewness': trade_skew,
            'Trade_Kurtosis': trade_kurt,
            'Trade_Density': trade_density
        }
    
    # Extract features for all accounts
    feature_data = []
    for account, trader_data in df.groupby('Account'):
        features = extract_features(trader_data)
        features['Account'] = account
        feature_data.append(features)
    
    feature_df = pd.DataFrame(feature_data)
    
    # Handle missing/infinite values - replace with median of the column
    numeric_cols = feature_df.select_dtypes(include=np.number).columns
    for col in numeric_cols:
        feature_df[col] = feature_df[col].replace([np.inf, -np.inf], np.nan)
        median_val = feature_df[col].median()
        feature_df[col] = feature_df[col].fillna(median_val)
    
    # Standardize features
    scaler = StandardScaler()
    features_to_scale = feature_df.drop('Account', axis=1)
    scaled_features = scaler.fit_transform(features_to_scale)
    
    # Perform clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    feature_df['Cluster'] = kmeans.fit_predict(scaled_features)
    
    # Add PCA components for visualization
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(scaled_features)
    feature_df['PC1'] = principal_components[:, 0]
    feature_df['PC2'] = principal_components[:, 1]
    
    return feature_df

def plot_clusters(feature_df):
    fig = px.scatter(
        feature_df, 
        x='PC1', 
        y='PC2', 
        color='Cluster',
        hover_name='Account',
        title='Trader Strategy Clusters',
        labels={'PC1': 'Strategy Pattern Component 1', 'PC2': 'Strategy Pattern Component 2'},
        hover_data=feature_df.columns
    )
    fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')))
    fig.show()

In [52]:
clustered_data=cluster_traders(df,n_clusters=7)

c:\Users\Abhi9\.jupyter\conda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.



In [53]:
clustered_data['Cluster'].value_counts().sort_values(ascending=False)

Cluster
6    41
4    27
5    13
0     6
2     6
1     6
3     1
Name: count, dtype: int64

In [57]:
df[df['Account'].isin(clustered_data[clustered_data['Cluster']==6]['Account'].to_list())]

,Unnamed: 0,Account,Timestamp IST,Size USD,Closed PnL,Coin,Cumulative PnL,Month
2774643,7690795,0x0087188d27ac73b715ad38dff9b99554861a2b55,2025-01-12 16:12:44,20113.64,0.0,HYPE,0.000000,2025-01
2774644,7690796,0x0087188d27ac73b715ad38dff9b99554861a2b55,2025-01-12 16:18:10,7090.03,0.0,HYPE,0.000000,2025-01
2774645,7690797,0x0087188d27ac73b715ad38dff9b99554861a2b55,2025-01-12 16:18:26,2646.35,0.0,HYPE,0.000000,2025-01
2774646,7690798,0x0087188d27ac73b715ad38dff9b99554861a2b55,2025-01-12 16:18:26,1186.20,0.0,HYPE,0.000000,2025-01
2774647,7690799,0x0087188d27ac73b715ad38dff9b99554861a2b55,2025-01-12 16:18:28,1769.96,0.0,HYPE,0.000000,2025-01
...,...,...,...,...,...,...,...,...
2246822,6368146,0xfef198f2d9782ca670e64a988fbbcfd3f1f15563,2025-06-30 16:51:14,11.51,0.0,PENGU,146525.146499,2025-06
2246823,6368147,0xfef198f2d9782ca670e64a988fbbcfd3f1f15563,2025-06-30 16:51:14,11.51,0.0,PENGU,146525.146499,2025-06
2246824,6368148,0xfef198f2d9782ca670e64a988fbbcfd3f1f15563,2025-06-30 16:51:14,97.37,0.0,PENGU,146525.146499,2025-06
2246825,6368149,0xfef198f2d9782ca670e64a988fbbcfd3f1f15563,2025-06-30 16:51:14,122.26,0.0,PENGU,146525.146499,2025-06


In [61]:
merged_df=df.merge(right=clustered_data,on='Account')

In [62]:
merged_df.columns

Index(['Unnamed: 0', 'Account', 'Timestamp IST', 'Size USD', 'Closed PnL',
       'Coin', 'Cumulative PnL', 'Month', 'Cumulative_PnL', 'Timeline_Length',
       'Trend_Stability', 'Max_Drawdown', 'Recovery_Factor', 'Win_Rate',
       'Profit_Factor', 'Trade_Skewness', 'Trade_Kurtosis', 'Trade_Density',
       'Cluster', 'PC1', 'PC2'],
      dtype='object')

In [64]:
merged_df.drop(columns='Unnamed: 0',axis=1,inplace=True)

In [65]:
merged_df.columns

Index(['Account', 'Timestamp IST', 'Size USD', 'Closed PnL', 'Coin',
       'Cumulative PnL', 'Month', 'Cumulative_PnL', 'Timeline_Length',
       'Trend_Stability', 'Max_Drawdown', 'Recovery_Factor', 'Win_Rate',
       'Profit_Factor', 'Trade_Skewness', 'Trade_Kurtosis', 'Trade_Density',
       'Cluster', 'PC1', 'PC2'],
      dtype='object')

In [70]:
for i in  range(0,7):
    print(i)
    merged_df[merged_df['Cluster']==i][['Account', 'Timestamp IST', 'Size USD', 'Closed PnL', 'Coin',
        'Cumulative PnL',
        'Trend_Stability', 'Max_Drawdown', 'Recovery_Factor', 'Win_Rate',
        'Profit_Factor', 'Trade_Skewness', 'Trade_Kurtosis', 'Trade_Density']].to_csv(f"Cluster {i}.csv")

0
1
2
3
4
5
6


In [73]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5875990 entries, 0 to 5875989
Data columns (total 20 columns):
 #   Column           Dtype         
---  ------           -----         
 0   Account          object        
 1   Timestamp IST    datetime64[ns]
 2   Size USD         float64       
 3   Closed PnL       float64       
 4   Coin             object        
 5   Cumulative PnL   float64       
 6   Month            period[M]     
 7   Cumulative_PnL   float64       
 8   Timeline_Length  int64         
 9   Trend_Stability  float64       
 10  Max_Drawdown     float64       
 11  Recovery_Factor  float64       
 12  Win_Rate         float64       
 13  Profit_Factor    float64       
 14  Trade_Skewness   float64       
 15  Trade_Kurtosis   float64       
 16  Trade_Density    float64       
 17  Cluster          int32         
 18  PC1              float64       
 19  PC2              float64       
dtypes: datetime64[ns](1), float64(14), int32(1), int64(1), object(2), pe

In [ ]:
import plotly.express as px # interactable plot
import matplotlib.pyplot as plt
clusters=merged_df['Cluster'].astype(str)


MemoryError: 

<Figure size 2500x2000 with 0 Axes>

In [ ]:
plt.figure(figsize=(25,20))
px.scatter(y=merged_df['Closed PnL'],x=merged_df['Timestamp IST'].dt.month,color=clusters,color_discrete_sequence=px.colors.qualitative.G10).show()

In [14]:
def cluster_all_samples(df, n_clusters=4, lookback_window=10):
    """
    Cluster ALL samples while preserving original DataFrame structure
    
    Parameters:
    df (pd.DataFrame): Input trade data
    n_clusters (int): Number of clusters
    lookback_window (int): Trades to consider for local features
    
    Returns:
    pd.DataFrame: Original df with added cluster labels and features
    """
    # Preprocess
    df = df.copy()
    df['Closed PnL'] = df['Closed PnL'].replace('-', np.nan).astype(float)
    df['Timestamp IST'] = pd.to_datetime(df['Timestamp IST'])
    df = df.sort_values(['Account', 'Timestamp IST'])
    
    # Feature engineering for EACH trade
    features = []
    for account, trader_data in df.groupby('Account'):
        trader_data = trader_data.copy()
        
        # Rolling features
        trader_data['Cumulative_PnL'] = trader_data['Closed PnL'].cumsum()
        trader_data['Rolling_Mean'] = trader_data['Closed PnL'].rolling(window=lookback_window).mean()
        trader_data['Rolling_Std'] = trader_data['Closed PnL'].rolling(window=lookback_window).std()
        trader_data['PnL_Trend'] = trader_data['Closed PnL'].rolling(window=lookback_window).apply(
            lambda x: np.polyfit(range(len(x)), x, 1)[0], raw=True
        )
        
        # Recent performance
        trader_data['Recent_Win_Rate'] = trader_data['Closed PnL'].gt(0).rolling(window=lookback_window).mean()
        trader_data['Recent_Profit_Factor'] = trader_data['Closed PnL'].rolling(window=lookback_window).apply(
            lambda x: x[x>0].sum()/abs(x[x<0].sum()) if x[x<0].sum()<0 else 1, raw=True
        )
        
        features.append(trader_data)
    
    # Combine all traders
    feature_df = pd.concat(features).reset_index(drop=True)
    
    # Select feature columns for clustering
    feature_cols = [
        'Closed PnL', 'Cumulative_PnL', 'Rolling_Mean', 
        'Rolling_Std', 'PnL_Trend', 'Recent_Win_Rate',
        'Recent_Profit_Factor'
    ]
    
    # Clean and scale
    X = feature_df[feature_cols].fillna(0)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Cluster ALL samples
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    feature_df['Cluster'] = kmeans.fit_predict(X_scaled)
    
    return feature_df

In [15]:
clustered_df=cluster_all_samples(df)

In [17]:
clustered_df.isnull().sum()

Unnamed: 0                0
Account                   0
Timestamp IST             0
Size USD                  0
Closed PnL                0
Coin                      0
Cumulative_PnL            0
Rolling_Mean            900
Rolling_Std             900
PnL_Trend               900
Recent_Win_Rate         900
Recent_Profit_Factor    900
Cluster                   0
dtype: int64

In [19]:
clustered_df['Cluster'].value_counts()

Cluster
3    4198260
2    1614536
0      63193
1          1
Name: count, dtype: int64

In [22]:
df=pd.read_csv(r"C:\Users\Abhi9\Downloads\simple_straight.csv")
df=df.drop(labels=3098880,axis=0)

C:\Users\Abhi9\AppData\Local\Temp\ipykernel_32756\3482538797.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"C:\Users\Abhi9\Downloads\simple_straight.csv")


In [ ]:
total_df=cluster_traders(df)